In [ ]:
import helper_functions as hf

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torchinfo import summary

from pathlib import Path
import numpy as np

import random
import PIL
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
import time

REBUILD_DATA = False

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Get data

In [ ]:
data_path = Path("/data")

In [ ]:
hf.walk_through_dir(data_path)

In [ ]:
path_list = list(data_path.glob('*/*/*.jpg'))
len(path_list)

In [ ]:
PIL.Image.open(random.choice(path_list))

# Preparing the data

In [ ]:
COLOR_CHANNELS = 3
IMG_SIZE = 128

Image Normalization

The values of mean and standard deviation of the training set are [0.4883, 0.4553, 0.4170],[0.2229, 0.2182, 0.2185]

Run the below cell if you want to calculate the mean and standard deviation again.

In [ ]:
# # Run this cell to calculate mean and deviation of color
# # Calculate mean and deviation of color
# data_tranformer = transforms.Compose([
#     transforms.Resize((IMG_SIZE, IMG_SIZE)),
#     transforms.ToTensor()
# ])

# train_dataset = datasets.ImageFolder(root=data_path / "train", transform=data_tranformer)
# test_dataset = datasets.ImageFolder(root=data_path / "test", transform=data_tranformer)
# complete_dataset = torch.utils.data.ConcatDataset([train_dataset, test_dataset])

# # Calculate mean image
# mean_color = torch.zeros(COLOR_CHANNELS)
# std_color = torch.zeros(COLOR_CHANNELS)

# for image, _ in tqdm(complete_dataset):
#     mean_color += image.mean(dim=(1,2)) # Mean of each channel
#     std_color += image.std(dim=(1,2)) # Std of each channel
    
# mean_color = mean_color / len(complete_dataset) # Mean of all images
# std_color = std_color / len(complete_dataset) # Std of all images

# print(f"Mean color: {mean_color}")
# print(f"Std color: {std_color}")

In [ ]:
normalized_data_tranformer = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.4883, 0.4553, 0.4170],[0.2229, 0.2182, 0.2185]),
])

train_dataset = datasets.ImageFolder(root=data_path / "train",transform=normalized_data_tranformer)
test_dataset = datasets.ImageFolder(root=data_path / "test", transform=normalized_data_tranformer)

In [ ]:
train_dataset[0][0].shape, train_dataset[0][1]

In [ ]:
CLASSES = train_dataset.classes
CLASSES

After normalization pixel values are in the range of [-1, 1]. Hence the images have bad contrast.

In [ ]:
image, label = random.choice(train_dataset)
print(image.shape, CLASSES[label])
plt.imshow(image.permute(1, 2, 0), cmap='gray')
plt.axis('off')

In [ ]:
# Creat data loader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Creating Model

In [ ]:
# Create model
class TinyVGG(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(10)
        )
        
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), 
            nn.BatchNorm2d(10)
        )
        
        self.conv_block_3 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), 
            nn.BatchNorm2d(10)
        )
        
        self.Classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2560, 2)
        )
                    
    def forward(self, x):
        return self.Classifier(self.conv_block_3(self.conv_block_2(self.conv_block_1(x))))
    
    
class CatVsDog_2M(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.BatchNorm2d(256)
        )
        
        self.Classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
            # nn.Softmax(dim=1)
        )
                    
    def forward(self, x):
        return self.Classifier(self.conv_block(x))

In [ ]:
model = TinyVGG()
summary(model, input_size=(1, 3, 128, 128))

# Training Model

Overfitting to a single batch of data is good practice to check if the model is working properly.
If model can't overfit a single batch then there is something wrong with the model.

Run for about 10 epochs and final loss must be less than 0.1

In [ ]:
# Overfit on a single batch
overfit_model = CatVsDog_2M().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
overfit_model_optimizer = torch.optim.Adam(overfit_model.parameters(), lr=0.0005)
final_loss = hf.overfit_single_batch(overfit_model, train_loader, loss_fn, overfit_model_optimizer, epochs=10)

Create the model

In [ ]:
model = model.to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [ ]:
# Config
print("Using device:" + device)

EPOCHS = 10
test_every = 5

train_losses = []
train_accuracies = []

In [ ]:
# Trigger training
train_losses, test_losses = hf.train_model(model, train_loader, test_loader, loss_fn, optimizer, device, EPOCHS, test_every=test_every)

In [ ]:
# Plot loss curve
hf.plot_loss_curve(train_losses, test_losses, every_n=test_every)

In [ ]:
# Save model
torch.save(model.state_dict(), f"model_cats_vs_dogs_1M.pth")

In [ ]:
# Load model
# model.load_state_dict(torch.load("model_cats_vs_dogs_1M.pth"))

# Testing Model

In [ ]:
# Test Model
hf.show_random_prediction(model, test_dataset)

In [ ]:
predictions = hf.predict(model, test_dataset, device)

In [ ]:
hf.plot_metrics(test_dataset.targets, predictions, test_dataset.classes)